# prototype inference demo
Each step can be run separately.

## STEP 0: Import and read config

In [1]:
from preprocessing import full_prep
from config_submit import config as config_submit

import torch
from torch.nn import DataParallel
from torch.backends import cudnn
from torch.utils.data import DataLoader
from torch import optim
from torch.autograd import Variable

from layers import acc
from data_detector import DataBowl3Detector,collate
from data_classifier import DataBowl3Classifier

from utils import *
from split_combine import SplitComb
from test_detect import test_detect
from importlib import import_module
import pandas

datapath = config_submit['datapath']
prep_result_path = config_submit['preprocess_result_path']
skip_prep = config_submit['skip_preprocessing']
skip_detect = config_submit['skip_detect']

c:\users\hasee\appdata\local\programs\python\python35\lib\site-packages\dicom\__init__.py:53: UserWarning: 
This code is using an older version of pydicom, which is no longer 
maintained as of Jan 2017.  You can access the new pydicom features and API 
by installing `pydicom` from PyPI.
See 'Transitioning to pydicom 1.x' section at pydicom.readthedocs.org 
for more information.

  warnings.warn(msg)


## STEP 1: Pre-process stage1 / 2 data for inferencing.

In [ ]:
from preprocessing import full_prep
from config_submit import config as config_submit

datapath = config_submit['datapath']
prep_result_path = config_submit['preprocess_result_path']
skip_prep = config_submit['skip_preprocessing']
skip_detect = config_submit['skip_detect']

In [17]:
print('datapath:', datapath)
prep_result_path_demo = 'F:\\LargeFiles\\lfz\\prep_result_demo\\'
print('prep_result_path_demo:', prep_result_path_demo)

testsplit = full_prep(datapath,prep_result_path_demo,
                    n_worker = 1, # config_submit['n_worker_preprocessing']
                    use_existing=False) # config_submit['use_exsiting_preprocessing']

starting preprocessing
end preprocessing


## STEP 2: Load N-net

In [1]:
from preprocessing import full_prep
from config_submit import config as config_submit

import torch
from torch.nn import DataParallel
from torch.backends import cudnn
from torch.utils.data import DataLoader
from torch import optim
from torch.autograd import Variable

from layers import acc
from data_detector import DataBowl3Detector,collate
from data_classifier import DataBowl3Classifier

from utils import *
from split_combine import SplitComb
from test_detect import test_detect
from importlib import import_module
import pandas

datapath = config_submit['datapath']
prep_result_path = config_submit['preprocess_result_path']
skip_prep = config_submit['skip_preprocessing']
skip_detect = config_submit['skip_detect']

c:\users\hasee\appdata\local\programs\python\python35\lib\site-packages\dicom\__init__.py:53: UserWarning: 
This code is using an older version of pydicom, which is no longer 
maintained as of Jan 2017.  You can access the new pydicom features and API 
by installing `pydicom` from PyPI.
See 'Transitioning to pydicom 1.x' section at pydicom.readthedocs.org 
for more information.

  warnings.warn(msg)


In [2]:
nodmodel = import_module(config_submit['detector_model'].split('.py')[0])
config1, nod_net, loss, get_pbb = nodmodel.get_model()
checkpoint = torch.load(config_submit['detector_param'])
nod_net.load_state_dict(checkpoint['state_dict'])

In [6]:
nod_net= nod_net.cuda()

In [7]:
next(nod_net.parameters()).is_cuda

True

### Skip prep
Load the preprocessed npy.

In [5]:
testsplit = os.listdir(datapath)
testsplit

['0015ceb851d7251b8f399e39779d1e7d', '0030a160d58723ff36d73f41b170ec21']

In [6]:
bbox_result_path = './bbox_result'
if not os.path.exists(bbox_result_path):
    os.mkdir(bbox_result_path)

In [7]:
margin = 16 # TODO 32
sidelen = 64 # TODO 144
config1['datadir'] = prep_result_path
split_comber = SplitComb(sidelen,config1['max_stride'],config1['stride'],margin,pad_value= config1['pad_value'])

dataset = DataBowl3Detector(testsplit,config1,phase='test',split_comber=split_comber)
test_loader = DataLoader(dataset,batch_size = 1,
    shuffle = False,num_workers = 4,pin_memory=False,collate_fn =collate)

In [12]:
coord[0].shape

torch.Size([1, 100, 3, 24, 24, 24])

In [13]:
test_detect(test_loader, nod_net, get_pbb, bbox_result_path,config1,n_gpu=config_submit['n_gpu'])

torch.Size([100, 1, 96, 96, 96])


E:\SW_WS\github_SW\DSB2017-lfz-pytorch\test_detect.py:46: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  input = Variable(data[splitlist[i]:splitlist[i+1]], volatile = True) # TODO: no_grad() .cuda()
E:\SW_WS\github_SW\DSB2017-lfz-pytorch\test_detect.py:47: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  inputcoord = Variable(coord[splitlist[i]:splitlist[i+1]], volatile = True) # TODO .cuda()


[0, 'F:\\LargeFiles\\lfz\\prep_result_sub\\0015ceb851d7251b8f399e39779d1e7d']
torch.Size([100, 1, 96, 96, 96])
[1, 'F:\\LargeFiles\\lfz\\prep_result_sub\\0030a160d58723ff36d73f41b170ec21']
elapsed time is 858.38 seconds


## STEP3 :Load C-Net

In [1]:
from config_submit import config as config_submit
import numpy as np
import pandas
import torch
from torch.autograd import Variable
from torch.utils.data import DataLoader
from data_detector import DataBowl3Detector,collate
from importlib import import_module
import os
from data_classifier import DataBowl3Classifier


In [2]:

casemodel = import_module(config_submit['classifier_model'].split('.py')[0])
casenet = casemodel.CaseNet(topk=5)
checkpoint = torch.load(config_submit['classifier_param'])
casenet.load_state_dict(checkpoint['state_dict'])
casenet = casenet.cpu()

In [3]:
datapath = config_submit['datapath']
testsplit = os.listdir(datapath)
testsplit

['0015ceb851d7251b8f399e39779d1e7d', '0030a160d58723ff36d73f41b170ec21']

In [4]:
config2 = casemodel.config
bbox_result_path = './bbox_result'
if not os.path.exists(bbox_result_path):
    os.mkdir(bbox_result_path)

prep_result_path = config_submit['preprocess_result_path']
config2['bboxpath'] = bbox_result_path
config2['datadir'] = prep_result_path

In [5]:
filename = config_submit['outputfile']

def test_casenet(model,testset):
    data_loader = DataLoader(
        testset,
        batch_size = 1,
        shuffle = False,
        num_workers = 4,
        pin_memory=True)
    #model = model.cuda()
    model.eval()
    predlist = []

    #     weight = torch.from_numpy(np.ones_like(y).float().cuda()
    for i,(x,coord) in enumerate(data_loader):

        coord = Variable(coord) # TODO .cuda()
        x = Variable(x) # TODO .cuda()
        nodulePred,casePred,_ = model(x,coord)
        predlist.append(casePred.data.cpu().numpy())
        #print([i,data_loader.dataset.split[i,1],casePred.data.cpu().numpy()])
    predlist = np.concatenate(predlist)
    return predlist    
config2['bboxpath'] = bbox_result_path
config2['datadir'] = prep_result_path

In [ ]:
dataset = DataBowl3Classifier(testsplit, config2, phase = 'test')
predlist = test_casenet(casenet,dataset).T
df = pandas.DataFrame({'id':testsplit, 'cancer':predlist})
df.to_csv(filename,index=False)

## `DataBowl3Classifier` in `data_classifier.py`
Get top 5 for 'test' or sample 5 for training.

In [2]:
datapath = config_submit['datapath']

In [6]:
import os
testsplit = os.listdir(datapath)
testsplit

['0015ceb851d7251b8f399e39779d1e7d', '0030a160d58723ff36d73f41b170ec21']

In [11]:
from data_classifier import DataBowl3Classifier

dataset = DataBowl3Classifier(testsplit, config2, phase = 'test')

In [14]:
d0 = dataset[0]

In [15]:
len(d0)

2

### Input crop

In [16]:
d0[0].shape

torch.Size([5, 1, 64, 64, 64])

### coord
* 5, - top 5
* 3, - 
* 16, 
* 16,
* 16

In [17]:
d0[1].shape

torch.Size([5, 3, 16, 16, 16])

In [30]:
d0[1][0,:,:,:,:].max()

tensor(0.3312)

In [ ]:
coord = Variable(coord) # TODO .cuda()
x = Variable(x) # TODO .cuda()

In [32]:
#         xlist: n x k x 1x 96 x 96 x 96
#         coordlist: n x k x 3 x 24 x 24 x 24
xsize = xlist.size()
corrdsize = coordlist.size()
# Reshape the tensor to remove the 3rd axis.
xlist = xlist.view(-1,xsize[2],xsize[3],xsize[4],xsize[5])
coordlist = coordlist.view(-1,corrdsize[2],corrdsize[3],corrdsize[4],corrdsize[5])
casenet.NoduleNet()

Net(
  (preBlock): Sequential(
    (0): Conv3d(1, 24, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
    (1): BatchNorm3d(24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace)
    (3): Conv3d(24, 24, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
    (4): BatchNorm3d(24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace)
  )
  (forw1): Sequential(
    (0): PostRes(
      (conv1): Conv3d(24, 32, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
      (bn1): BatchNorm3d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Conv3d(32, 32, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
      (bn2): BatchNorm3d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (shortcut): Sequential(
        (0): Conv3d(24, 32, kernel_size=(1, 1, 1), stride=(1, 1, 1))
        (1): BatchNorm3d(32, eps=1e-05, 

In [33]:
prep_result_path

'F:\\LargeFiles\\lfz\\prep_result_sub\\'

In [6]:
import os
os.path.split( './sss/0015ceb851d7251b8f399e39779d1e7d')[-1]

'0015ceb851d7251b8f399e39779d1e7d'